In [18]:
import pandas as pd

from thresh import load_interface
import git, os, shutil, getpass
from github import Github

In [19]:
df = pd.read_csv("/home/li.mil/hierarchical-mds/human_eval/csv/preference_ranking.csv")
df

,label,output
0,"At present, there is insufficient RCT evidence...",Output 1: The provided snippets suggest that b...
1,There is inadequate evidence to draw strong co...,Output 1: The evidence suggests that various i...
2,Collaborative care is associated with signific...,Output 1: The articles provide evidence that c...
3,Administration of systemic prophylactic antibi...,Output 1: The evidence suggests that antibioti...
4,The limited available evidence suggests folate...,Output 1: The articles suggest that folate sup...
...,...,...
465,There was evidence from three pooled RCTs that...,"Output 1: The evidence from in vitro, animal s..."
466,Antibiotic prophylaxis is effective in prevent...,Output 1: The evidence from the provided snipp...
467,The practice of surgical and chemical sympathe...,Output 1: The evidence suggests that sympathec...
468,There is not sufficient evidence to support or...,Output 1: The evidence from the provided artic...


In [20]:
df.rename(columns={'label':'source', 'output': 'target'}, inplace=True)

In [21]:
df

,source,target
0,"At present, there is insufficient RCT evidence...",Output 1: The provided snippets suggest that b...
1,There is inadequate evidence to draw strong co...,Output 1: The evidence suggests that various i...
2,Collaborative care is associated with signific...,Output 1: The articles provide evidence that c...
3,Administration of systemic prophylactic antibi...,Output 1: The evidence suggests that antibioti...
4,The limited available evidence suggests folate...,Output 1: The articles suggest that folate sup...
...,...,...
465,There was evidence from three pooled RCTs that...,"Output 1: The evidence from in vitro, animal s..."
466,Antibiotic prophylaxis is effective in prevent...,Output 1: The evidence from the provided snipp...
467,The practice of surgical and chemical sympathe...,Output 1: The evidence suggests that sympathec...
468,There is not sufficient evidence to support or...,Output 1: The evidence from the provided artic...


In [22]:
df.head()

,source,target
0,"At present, there is insufficient RCT evidence...",Output 1: The provided snippets suggest that b...
1,There is inadequate evidence to draw strong co...,Output 1: The evidence suggests that various i...
2,Collaborative care is associated with signific...,Output 1: The articles provide evidence that c...
3,Administration of systemic prophylactic antibi...,Output 1: The evidence suggests that antibioti...
4,The limited available evidence suggests folate...,Output 1: The articles suggest that folate sup...


In [23]:
df.sample(10).to_json("cochrane.json", orient='records')

In [24]:
interface = load_interface('interface.yml')

In [25]:
local_template_path = "interface.yml"
local_data_path = "cochrane.json"
repository_name = "prolific-deployment-ml"

In [26]:
if os.path.exists('.git_credentials'):
    # This will read the API token from a local 
    github_username = "datalabNU"
    github_token = open('.git_credentials', 'r').read()
else:
    github_username = input("Please enter your GitHub username: ")
    github_token = getpass.getpass("Please enter your GitHub access token (register a token at github.com/settings/tokens): ")

# Log into GitHub
g = Github(github_token)
user = g.get_user()

In [13]:
try:
    remote_repo = g.get_repo(f"{github_username}/{repository_name}").clone_url
    print(f"Found existing repository: {remote_repo}")
except Exception as e:
    remote_repo = user.create_repo(repository_name).clone_url
    print(f"Created new repository: {remote_repo}")

if not os.path.exists(repository_name):
    git.Repo.clone_from(remote_repo, repository_name)

repo = git.Repo(repository_name)


Found existing repository: https://github.com/datalabNU/prolific-deployment-ml.git


In [14]:
for file_path in [local_template_path, local_data_path]:
    template_repo_path = os.path.join(os.getcwd(), repository_name, file_path)
    shutil.copy(file_path, template_repo_path)
    repo.index.add([template_repo_path])
repo.index.commit("Add a new interface template and data")

origin = repo.remote("origin")

In [28]:
origin.push()
print(f"Pushed changes to {remote_repo}")

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git push --porcelain -- origin
  stderr: 'fatal: could not read Username for 'https://github.com': No such device or address'

In [18]:
template_path = f"{github_username}/{repository_name}/master/{local_template_path}"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: datalabNU/prolific-deployment/master/interface.yml
Hosted data: datalabNU/prolific-deployment/master/quarel.json


In [19]:
study_url = f"https://thresh.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

In [20]:
print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://thresh.tools/?gh=datalabNU/prolific-deployment/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}
Your study (with data) will be linked to: https://thresh.tools/?gh=datalabNU/prolific-deployment/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=datalabNU/prolific-deployment/master/quarel.json
